<a href="https://colab.research.google.com/github/ByungjunKim/FoucaultKoreanLiterature/blob/main/%5BColab%5D%ED%91%B8%EC%BD%94%EC%9D%B8%EC%9A%A9%EB%85%BC%EB%AC%B8_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 푸코 인용 논문 분석

In [ ]:
import pandas as pd
import ast
# matplotlib 폰트설정
import matplotlib.pyplot as plt
# https://github.com/killiansheriff/LovelyPlots
# plt.style.use(['ipynb','use_mathtext','colors5-light'])
# plt.rc('font', family='Malgun Gothic')
# print(plt.rcParams['font.family'])
plt.rcParams["figure.dpi"] = 240 # DPI 고화질로 향상

from scipy.stats import chi2_contingency
from scipy.stats import fisher_exact
from itertools import combinations
from statsmodels.sandbox.stats.multicomp import MultiComparison
from statsmodels.stats.multitest import multipletests
import scipy.stats

In [ ]:
# github clone
!git clone https://github.com/ByungjunKim/FoucaultKoreanLiterature.git
!mkdir stats

In [ ]:
# 시각화 필요 라이브러리 불러오기
import matplotlib.font_manager as fm
import os
import matplotlib.pyplot as plt

# apt-get : 패키지 설치 명령어
# -qq : 에러외의 메세지 숨기는 옵션키
!apt-get -qq install fonts-nanum


fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 12, 'font.family': 'NanumGothic'}) # 폰트 설정

# 테스트 해보기
fig = plt.figure(figsize=(10, 10))
fig, ax = plt.subplots(subplot_kw=dict(projection='3d'), figsize=(10, 10))

graph_title = '시험'
ax.set_title(f'{graph_title}')
ax.set_xlabel('야호 (Hz)', labelpad=20)
ax.set_ylabel('시간 (s)', labelpad=20)
ax.set_zlabel('이건 뭐지 (dB/Hz)', labelpad=20)

ax.view_init(20, 20)
plt.show()

### 1. 어떤 푸코의 저작이 인용되었나?

In [ ]:
df = pd.read_pickle('./FoucaultKoreanLiterature/231005_2008_2021_df.pkl') # 한국현대문학 논문 서지정보
ref_df = pd.read_pickle('./FoucaultKoreanLiterature/231005_2008_2021_ref_df.pkl') # 한국현대문학 논문 참고문헌
ref_fu = pd.read_pickle('./FoucaultKoreanLiterature/231005_2008_2021_ref_fu.pkl') # 푸코 인용 논문 참고문헌

In [ ]:
# 전체 분석 대상 논문 게재 추이
df['year'].value_counts().sort_index().plot()

In [ ]:
# 푸코 인용 추이
print(ref_fu['year'].value_counts().sort_index())
ref_fu['year'].value_counts().sort_index().plot()

In [ ]:
ref_fu['classification'].value_counts(normalize=True)

In [ ]:
# pd.DataFrame(ref_fu['uri'].value_counts()).reset_index().to_excel('./stats/푸코저작_인용.xlsx')

In [ ]:
pd.crosstab(ref_fu['classification'],ref_fu['year'],normalize='columns')

In [ ]:
pd.crosstab(ref_fu['classification'],ref_fu['year'],normalize='columns').T.plot(style=['-', '--', ':', '-.'])
# 범례 위치 설정: 플롯의 오른쪽 바깥쪽
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

# 축 조정 (범례에 의해 가려지는 부분 방지)
plt.subplots_adjust(right=0.75)

In [ ]:
# 연도별 주요 저작 인용 패턴
pd.crosstab(ref_fu['uri'],ref_fu['year'],margins=True).to_excel('./stats/연도별_푸코저작_인용.xlsx')

In [ ]:
# 성별 주요 저작 인용 패턴
pd.crosstab(ref_fu['uri'],ref_fu['gender'],margins=True).to_excel('./stats/성별_푸코저작_인용.xlsx')

In [ ]:
top10 = pd.read_excel('./FoucaultKoreanLiterature/stats/연도별_푸코저작_인용_Top10.xlsx')
top10

In [ ]:
# Simplifying the line styles to ensure compatibility
line_styles = ['-', '--', '-.', ':', (0, (3, 10, 1, 10))]

# Re-plotting
plt.figure(figsize=(15, 8))

for i, title in enumerate(top10['title']):
    plt.plot(top10.columns[1:], top10.iloc[i, 1:], linestyle=line_styles[i % len(line_styles)], label=title)

# plt.title('연도별 미셸 푸코 저작 인용 추이')
plt.xlabel('연도')
plt.ylabel('인용 횟수')
plt.xticks(rotation=45)
plt.legend(title='작품', loc='upper left')
plt.grid(True)

# Improving font properties for better display of Korean characters
# plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['font.size'] = 12
# 범례 위치 설정: 플롯의 오른쪽 바깥쪽
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

# 축 조정 (범례에 의해 가려지는 부분 방지)
plt.subplots_adjust(right=0.75)
plt.show()

### 2. 누가 인용하는가?

##### 세대

In [ ]:
df.loc[~pd.isna(df['birth']),'birth'] = df[~pd.isna(df['birth'])]['birth'].map(lambda x:1900 + int(x))
df['birth']

In [ ]:
# 생년 세대
df.loc[~pd.isna(df['birth']),'birth_gen'] = \
pd.cut(df.loc[~pd.isna(df['birth']),'birth'],bins=[1900,1940,1950,1960,1970,1980,1990,2000],labels=['40년생 이전','40년대생','50년대생','60년대생','70년대생','80년대생','90년생 이후'],right=False)

In [ ]:
# 생년 세대 merge
ref_fu = ref_fu.merge(df[['artiId','birth_gen']])

In [ ]:
# 세대별 주요 저작 인용 패턴
pd.crosstab(ref_fu['uri'],ref_fu['birth_gen'],margins=True).to_excel('./stats/세대별_푸코저작_인용.xlsx')

In [ ]:
df['birth_gen'].value_counts()

In [ ]:
df['birth_gen'].value_counts(normalize=True)

In [ ]:
pd.crosstab(df['birth_gen'],df['mf'])

In [ ]:
pd.crosstab(df['birth_gen'],df['mf'],normalize='index')

In [ ]:
# Continggency Table 생성
birth_mf = pd.crosstab(df['birth_gen'],df['mf'])
birth_mf = birth_mf[1:]
birth_mf.columns = ['푸코_비인용','푸코_인용']
with pd.ExcelWriter(f'./stats/푸코_인용_인구사회정보.xlsx',mode='w',engine='openpyxl') as writer:
    (birth_mf).to_excel(writer,sheet_name='세대-푸코 Table')
birth_mf

In [ ]:
chi2_contingency(birth_mf)

In [ ]:
# 50 ~ 80년대생만
chi2_contingency(birth_mf[1:-1])

##### 성별

In [ ]:
df['gender'].value_counts()

In [ ]:
# Continggency Table 생성
gender_mf = pd.crosstab(df['gender'],df['mf'])
gender_mf.columns = ['푸코_비인용','푸코_인용']
with pd.ExcelWriter('./stats/푸코_인용_인구사회정보.xlsx',mode='a',engine='openpyxl') as writer:
    (gender_mf).to_excel(writer,sheet_name='성별-푸코 Table')

In [ ]:
gender_mf

In [ ]:
chi2_contingency(gender_mf)

In [ ]:
pd.crosstab(df['gender'],df['mf'],normalize='index')

##### 김윤식

In [ ]:
# 논문 게재 year 컬럼 추가
ref_df = ref_df.merge(df[['artiId','year','gender']])

In [ ]:
ref_kim = ref_df.loc[ref_df['author']=='김윤식'].reset_index(drop=True)

In [ ]:
pd.crosstab(df['year'],df['gender'])

In [ ]:
pd.crosstab(df['year'],df['gender'],normalize='index')